In [ ]:
pip install -U langchain langgraph langchain_openai elasticsearch streamlit langchain-community sentence_transformers

In [1]:
import getpass
import os


def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:········


In [21]:
from langchain.embeddings.openai import OpenAIEmbeddings
from elasticsearch import Elasticsearch
import pandas as pd

# Connect to ElasticSearch
es_client = Elasticsearch("http://localhost:9200")

# Initialize embeddingd
# embeddings = OpenAIEmbeddings()

# Load data in dfs
claims_df = pd.read_csv("claims.csv")
mainframe_df = pd.read_csv("mainframe_data.csv")
vsps_df = pd.read_csv("vsps.csv")

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader

# Process data of claims
doc_loader_claims = DataFrameLoader(claims_df, page_content_column="Claim_ID")
claims_documents = doc_loader_claims.load()
claims_text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
claims_docs = claims_text_splitter.split_documents(claims_documents)

# Process data of mainframe
doc_loader_mainframe = DataFrameLoader(mainframe_df, page_content_column="Transaction_ID")
mainframe_documents = doc_loader_mainframe.load()
mainframe_text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
mainframe_docs = mainframe_text_splitter.split_documents(mainframe_documents)

# Process data of vsps
doc_loader_vsps = DataFrameLoader(vsps_df, page_content_column="Transaction_ID")
vsps_documents = doc_loader_vsps.load()
vsps_text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
vsps_docs = vsps_text_splitter.split_documents(vsps_documents)

In [4]:
from langchain.vectorstores import ElasticsearchStore
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

# Load an open-source embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

index_claims = ElasticsearchStore.from_documents(claims_docs, index_name="claims", embedding=embedding_model, es_connection=es_client)



# Load elasticsearch indices
# index_claims = ElasticsearchStore.from_documents(claims_docs, index_name="claims", embedding=embeddings, es_connection=es_client)
# index_visa_stop_payment = ElasticsearchStore.from_documents(vsps_docs, index_name="visa_stop_payment", embedding=embeddings, es_connection=es_client)
# index_mainframe = ElasticsearchStore.from_documents(mainframe_docs, index_name="mainframe_transactions", embedding=embeddings, es_connection=es_client)

C:\Users\Prathamesh Bonde\AppData\Local\Temp\ipykernel_36224\3354538367.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [23]:
from datetime import datetime

def format_metadata(metadata):
    if "Placement_Date" in metadata:
        date_value = metadata["Placement_Date"]
        
        # Replace "-" with None or "N/A"
        if date_value == "-":
            metadata["Placement_Date"] = None  # Or use "N/A" if storing as string
        else:
            try:
                # Convert to ISO format (YYYY-MM-DD) if valid
                metadata["Placement_Date"] = datetime.strptime(date_value, "%d-%m-%Y").strftime("%Y-%m-%d")
            except ValueError:
                print(f"Skipping invalid date format: {date_value}")
                metadata["Placement_Date"] = None  # Handle unexpected formats
                
    return metadata

# Apply this function to each document
for doc in vsps_docs:
    doc.metadata = format_metadata(doc.metadata)


In [25]:
index_visa_stop_payment = ElasticsearchStore.from_documents(vsps_docs, index_name="visa_stop_payment", embedding=embedding_model, es_connection=es_client)

In [26]:
index_mainframe = ElasticsearchStore.from_documents(mainframe_docs, index_name="mainframe_transactions", embedding=embedding_model, es_connection=es_client)

In [40]:
index_visa_stop_payment.similarity_search("TXN002", k=1)[0].metadata.get("stop_payment_placed", "No")

'No'

In [32]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
import json

# Initialise LLM
llm = ChatOpenAI(temperature=0, streaming=True)

# Define AgentState
class AgentState(dict):
    def ___int___(self, claim_id=None, transaction_id=None, search_results=None):
        super().___init___()
        self["claim_id"] = claim_id
        self["transaction_id"] = transaction_id
#         self["search_results"] = search_results or {}

In [41]:
def agent(state):
    claim_id = state["claim_id"]
    
    # Fetch claim metadata to get transaction id
    claim_metadata = index_claims.similarity_search(claim_id, k=1)
    transaction_id = claim_metadata[0].metadata.get("transaction_id", "UNKNOWN")
    
    state["transaction_id"] = transaction_id
    
    return state

def supervisor(state):
    
#     claim_id = state["claim_id"]
    
#     # Fetch claim metadata to get transaction id
#     claim_metadata = index_claims.similarity_search(claim_id, k=1)
#     transaction_id = claim_metadata[0].metadata.get("transaction_id", "UNKNOWN")
    
#     state["transaction_id"] = transaction_id
    
    # Query Visa Stop Payment Service to check if Stop Payment was placed
    stop_payment_status = index_visa_stop_payment.similarity_search(state["transaction_id"], k=1)
    stop_payment_flag = stop_payment_status[0].metadata.get("stop_payment_placed", "No") if stop_payment_status else "No"
    
    print("Stop Payment Status",stop_payment_status)
    print("Stop Payment Flag",stop_payment_flag)
    
    if stop_payment_flag == "Yes":
        return ["claims", "visa_stop_payment", "mainframe"]
    else:
        return ["claims", "mainframe"]

In [9]:
def query_claims(state):
    """Query Claims7.0 Database."""
    results = index_claims.similarity_search(state["claim_id"], k=1)
    return {"search_results": {**state["search_results"], "claims": results}}

def query_visa_stop_payment(state):
    """Query Visa Stop Payment Service Database."""
    results = index_visa_stop_payment.similarity_search(state["transaction_id"], k=1)
    return {"search_results": {**state["search_results"], "visa_stop_payment": results}}

def query_mainframe(state):
    """Query Mainframe Transaction Database."""
    results = index_mainframe.similarity_search(state["transaction_id"], k=1)
    return {"search_results": {**state["search_results"], "mainframe": results}}

In [10]:
def aggregator(state):
    """Formats and summarizes stop payment validation results."""
    all_results = state["search_results"]

    structured_response = {
        "Claim ID": state["claim_id"],
        "Transaction ID": state["transaction_id"],
        "Claims Details": all_results.get("claims", []),
        "Visa Stop Payment Confirmation": all_results.get("visa_stop_payment", []),
        "Credit Card Transaction Data": all_results.get("mainframe", [])
    }

    summary_prompt = f"Summarize and validate stop payment claim:\n{json.dumps(structured_response, indent=2)}"
    summary = llm.invoke([HumanMessage(content=summary_prompt)])

    return {"messages": [AIMessage(content=summary.content)]}

In [42]:
from langgraph.graph import END, StateGraph, START
# Create a stateful graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", agent)
workflow.add_node("supervisor", supervisor)
workflow.add_node("claims", query_claims)
workflow.add_node("visa_stop_payment", query_visa_stop_payment)
workflow.add_node("mainframe", query_mainframe)
workflow.add_node("aggregator", aggregator)

workflow.add_edge(START, "agent")
workflow.add_edge("agent", "supervisor")

# Supervisor dynamically determines which data sources to query
workflow.add_conditional_edges(
    "supervisor",
    lambda state: supervisor(state),
    {
        "claims": "visa_stop_payment",
        "visa_stop_payment": "mainframe",
        "mainframe": "aggregator"
    }
)

# Aggregator is the final step
# workflow.set_entry_point("supervisor")

# Compile the graph
app = workflow.compile()

In [43]:
# Create an initial state with input values
initial_state = AgentState(
    claim_id="CLM002"
)

# Run the workflow with the input state
result = app.invoke(initial_state)

InvalidUpdateError: Must write to at least one of []